In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Mars
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат)
date_launch = Time("2028-12-10", scale="utc").tdb
date_flyby_Mars = Time("2029-07-15", scale="utc").tdb
date_flyby_Earth = Time("2031-03-30", scale="utc").tdb
date_arrival = Time("2033-03-13", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
mar = Ephem.from_body(Mars, time_range(date_launch, end=date_flyby_Mars))
earth_2 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_mar = Orbit.from_ephem(Sun, mar, date_flyby_Mars)
ss_earth_2 = Orbit.from_ephem(Sun, earth_2, date_flyby_Earth)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_mar)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_mar, ss_earth_2)
ss_trans_2, ss_target_2 = ss_mar.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_earth_2, ss_jup)
ss_trans_3, ss_target_3 = ss_earth_2.apply_maneuver(man_lambert_3, intermediate=True)

In [6]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=162 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=72 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=168 * u.deg), color="crimson", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [13]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=162 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [14]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=72 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [15]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=168 * u.deg), color="crimson", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [8]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_mar, v_mar = Ephem.from_body(Mars, date_flyby_Mars).rv(date_flyby_Mars)
r_earth_2, v_earth_2 = Ephem.from_body(Earth, date_flyby_Earth).rv(date_flyby_Earth)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [9]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

30691460.458745368 km
132036284.11701778 km
57239695.97479565 km

[-29.62788963   5.57696883   2.41796489] km / s


In [10]:
# Фазовые координаты Марса

print (f"{r_mar [0]}")
print (f"{r_mar [1]}")
print (f"{r_mar [2]}"'\n')

print (v_mar.to(u.km / u.s))

-122036834.00961111 km
-177435176.37022862 km
-78084358.46043485 km

[21.40118854 -9.64417278 -5.00093956] km / s


In [11]:
# Фазовые координаты Земли_2

print (f"{r_earth_2 [0]}")
print (f"{r_earth_2 [1]}")
print (f"{r_earth_2 [2]}"'\n')

print (v_earth_2.to(u.km / u.s))

-147873937.3832074 km
-20468687.52985262 km
-8859513.578281814 km

[  4.00585875 -27.12264703 -11.75568028] km / s


In [12]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

554844071.3171368 km
-467338165.042866 km
-213812427.97456494 km

[8.71575804 9.45553119 3.84067137] km / s


In [16]:
nh_tof_1 = date_flyby_Mars - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_mar), = iod.lambert(Sun.k, r_earth, r_mar, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=217.00000000437424>

In [17]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 3.05079864 km / s>

In [18]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [-2.89401149,  0.9626579 , -0.07320926] km / s>

In [19]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end = (norm(vk_mar - v_mar)).to(u.km / u.s)
V_inf_end

<Quantity 5.1290316 km / s>

In [20]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end_vec = (vk_mar - v_mar).to(u.km / u.s)
V_inf_end_vec

<Quantity [-4.45291988e+00, -2.54528334e+00,  1.52256103e-03] km / s>

In [21]:
nh_tof_2 = date_flyby_Earth - date_flyby_Mars # время полета Марс - Земля2

(v0_mar, vk_Ea2), = iod.lambert(Sun.k, r_mar, r_earth_2, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=623.0000000226305>

In [22]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_vec_0 = (norm(v0_mar - v_mar)).to(u.km / u.s)
V_inf_end_vec_0

<Quantity 14.300465 km / s>

In [23]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_vec_0_vec = (v0_mar - v_mar).to(u.km / u.s)
V_inf_end_vec_0_vec

<Quantity [-9.87886284, -9.71304656, -3.54515087] km / s>

In [24]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2 = (norm(vk_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2

<Quantity 12.41597665 km / s>

In [25]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2_vec = (vk_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [-11.69433637,  -3.74237749,  -1.84216825] km / s>

In [26]:
nh_tof_3 = date_arrival - date_flyby_Earth # время полета Земля2 - Юпитер

(V0_Ea2, Vk_Jup), = iod.lambert(Sun.k, r_earth_2, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=713.999999998646>

In [27]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_end_2_0 = (norm(V0_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2_0

<Quantity 12.69837302 km / s>

In [34]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_end_2_0_vec = (V0_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_end_2_0_vec

<Quantity [-9.26411223, -7.69047945, -4.03502511] km / s>

In [28]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 7.24962661 km / s>

In [29]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k_vec = (Vk_Jup - v_jup).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [-1.25752155, -6.65329167, -2.59025779] km / s>

In [30]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

<TimeDelta object: scale='tdb' format='jd' value=1554.0000000256507>

In [31]:
scalar_EM = r_earth[0]*r_mar[0] + r_earth[1]*r_mar[1] +r_earth[2]*r_mar[2] 
scalar_ME_2 = r_earth_2[0]*r_mar[0] + r_earth_2[1]*r_mar[1] +r_earth_2[2]*r_mar[2]
scalar_E2J = r_earth_2[0]*r_jup[0] + r_earth_2[1]*r_jup[1] +r_earth_2[2]*r_jup[2]

In [32]:
fi_EM = (scalar_EM/norm(r_earth)/norm(r_mar))
fi_ME2 = (scalar_ME_2/norm(r_earth_2)/norm(r_mar))
fi_E2J = (scalar_E2J/norm(r_earth_2)/norm(r_jup))

In [33]:
np.degrees(np.arccos(fi_EM))

<Quantity 159.84569246 deg>

In [34]:
np.degrees(np.arccos(fi_ME2))

<Quantity 49.23165462 deg>

In [35]:
np.degrees(np.arccos(fi_E2J))

<Quantity 128.61679564 deg>

In [36]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 3.0507986 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 3.51948851 km / s>

In [37]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 7.2496266

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

13.18628742035679
17.983782441612995


<Quantity -4.79749502 km / s>

In [38]:
## параметры участка Землия-Марс 

from math import*

to_rad = pi/180

ist_an_1 = 0.5257992 * to_rad
ist_an_2 = 160.3103259 * to_rad

exs_EM = (norm(r_mar)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_mar)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.22580193798061537
Фокальный параметр 180370384.37474722 km
Большая полуось 190060928.80962747 km


In [39]:
## параметры участка Марс-Земля_2 

ist_an_1_ME2 = 119.1533184 * to_rad
ist_an_2_ME2 = 70.1535905 * to_rad

exs_ME2 = (norm(r_earth_2)/norm(r_mar) - 1) / (cos(ist_an_1_ME2) - (norm(r_earth_2)/norm(r_mar) * cos(ist_an_2_ME2)))
p_ME2 = norm(r_mar)*(1+cos(ist_an_1_ME2)*exs_ME2)
a_ME2 = p_ME2/(1 - exs_ME2*exs_ME2)

print (f"Эксцентриситет {exs_ME2}")
print (f"Фокальный параметр {p_ME2}")
print (f"Большая полуось {a_ME2}")

Эксцентриситет 0.4897943947701658
Фокальный параметр 174413878.4327133 km
Большая полуось 229461314.98272356 km


In [40]:
## параметры участка Земля_2 - Юпитер

ist_an_1_E2J = 40.5301318 * to_rad 
ist_an_2_E2J = 168.5570527 * to_rad

exs_E2J = (norm(r_jup)/norm(r_earth_2) - 1) / (cos(ist_an_1_E2J) - (norm(r_jup)/norm(r_earth_2) * cos(ist_an_2_E2J)))
p_E2J = norm(r_earth_2)*(1+cos(ist_an_1_E2J)*exs_E2J)
a_E2J = p_E2J/(1 - exs_E2J*exs_E2J)

print (f"Эксцентриситет {exs_E2J}")
print (f"Фокальный параметр {p_E2J}")
print (f"Большая полуось {a_E2J}")

Эксцентриситет 0.7097058054787607
Фокальный параметр 230215200.55984646 km
Большая полуось 463846472.9918538 km


In [41]:
## Вектор гиперболического избытка скорости после грав. маневра у Марса

r_pi_Mar = 3735.8094
muMar = 42828

V_inf_Mar = 5.1290316
V_inf_Mar_vec_t_0 = -4.4529199
V_inf_Mar_vec_t_1 = -2.5452833
V_inf_Mar_vec_t_2 = 0.001522561


betta_Mar = 2*asin(1/(1+((r_pi_Mar*V_inf_Mar*V_inf_Mar)/muMar)))
gamma_Mar = pi
Vxy_inf_Mar = sqrt (V_inf_Mar_vec_t_0**2 + V_inf_Mar_vec_t_1**2)

V_grav_X_Mar = V_inf_Mar_vec_t_0 * cos(betta_Mar) - (V_inf_Mar_vec_t_1*V_inf_Mar)/Vxy_inf_Mar * sin(betta_Mar)*cos(gamma_Mar) - (V_inf_Mar_vec_t_0*V_inf_Mar_vec_t_2)/Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Y_Mar = V_inf_Mar_vec_t_1 * cos(betta_Mar) + (V_inf_Mar_vec_t_0*V_inf_Mar)/Vxy_inf_Mar * sin(betta_Mar)*cos(gamma_Mar) - (V_inf_Mar_vec_t_1*V_inf_Mar_vec_t_2)/Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Z_Mar = V_inf_Mar_vec_t_2 * cos(betta_Mar) + Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Mar = sqrt (V_grav_X_Mar**2 + V_grav_Y_Mar**2 + V_grav_Z_Mar**2) 

print (f"betta_Mar      {betta_Mar*180/pi} deg")
print (f"V_grav_X_Mar   {V_grav_X_Mar * u.km / u.s}")
print (f"V_grav_Y_Mar   {V_grav_Y_Mar * u.km / u.s}")
print (f"V_grav_Z_Mar   {V_grav_Z_Mar * u.km / u.s}")
print (f"V_grav_Mar     {V_grav_Mar * u.km / u.s}")

betta_Mar      35.33792086043422 deg
V_grav_X_Mar   -5.104677450523802 km / s
V_grav_Y_Mar   0.4992310644559299 km / s
V_grav_Z_Mar   0.0012420366873977964 km / s
V_grav_Mar     5.129031592051185 km / s


In [42]:
## Вектор гиперболического избытка скорости после грав. маневра у Земли_2

r_pi_Ear = 9950.7194
muEar = 398600

V_inf_Ear = 12.415977
V_inf_Ear_vec_t_0 = -11.694336
V_inf_Ear_vec_t_1 = -3.7423775
V_inf_Ear_vec_t_2 = -1.8421682


betta_Ear = 2*asin(1/(1+((r_pi_Ear*V_inf_Ear*V_inf_Ear)/muEar)))
gamma_Ear = pi
Vxy_inf_Ear = sqrt (V_inf_Ear_vec_t_0**2 + V_inf_Ear_vec_t_1**2)

V_grav_X_Ear = V_inf_Ear_vec_t_0 * cos(betta_Ear) - (V_inf_Ear_vec_t_1*V_inf_Ear)/Vxy_inf_Ear * sin(betta_Ear)*cos(gamma_Ear) - (V_inf_Ear_vec_t_0*V_inf_Ear_vec_t_2)/Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Y_Ear = V_inf_Ear_vec_t_1 * cos(betta_Ear) + (V_inf_Ear_vec_t_0*V_inf_Ear)/Vxy_inf_Ear * sin(betta_Ear)*cos(gamma_Ear) - (V_inf_Ear_vec_t_1*V_inf_Ear_vec_t_2)/Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Z_Ear = V_inf_Ear_vec_t_2 * cos(betta_Ear) + Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Ear = sqrt (V_grav_X_Ear**2 + V_grav_Y_Ear**2 + V_grav_Z_Ear**2) 

print (f"betta_Ear      {betta_Ear*180/pi} deg")
print (f"V_grav_X_Ear   {V_grav_X_Ear * u.km / u.s}")
print (f"V_grav_Y_Ear   {V_grav_Y_Ear * u.km / u.s}")
print (f"V_grav_Z_Ear   {V_grav_Z_Ear * u.km / u.s}")
print (f"V_grav_Ear     {V_grav_Ear * u.km / u.s}")

betta_Ear      23.805841934548738 deg
V_grav_X_Ear   -12.226838984008612 km / s
V_grav_Y_Ear   1.3491448067407155 km / s
V_grav_Z_Ear   -1.6854337980045437 km / s
V_grav_Ear     12.415976415001932 km / s
